In [1]:
import pandas as pd
import numpy as np
from pystoned import CNLS, CNLSDDF, StoNED
from pystoned.constant import CET_MULT, CET_ADDI, FUN_COST, FUN_PROD, RTS_VRS, RTS_CRS, RED_MOM, OPT_LOCAL, RED_QLE, RED_KDE

In [2]:
data = pd.read_csv("../../data_ready/data_ready.csv").dropna()

In [3]:
data_svk = data[data["CNT"] == 'SVK']

inputs_svk = data_svk[["total_students", "ratio_f2m", "ratio_t2s", "ratio_pt2ft"]].values.tolist()
outputs_svk = data_svk[["avg_math", "avg_cog", "avg_science", "avg_read"]]
d_outputs_svk = data_svk[["avg_math", "avg_cog", "avg_science", "avg_read"]].values.tolist()
undesirables_svk = data_svk[["dropout_rate"]].values.tolist()
contextuals_svk = data_svk[["bol_competition", "bol_location"]].values.tolist()
data_svk["avg_scores"] = outputs_svk.mean(axis = 1)
z_output_svk = data_svk[["avg_scores"]].values.tolist()

<ipython-input-3-3be619db88b4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_svk["avg_scores"] = outputs_svk.mean(axis = 1)


In [4]:
model1_svk = CNLS.CNLS(
    y = z_output_svk, 
    x = inputs_svk, 
    z = contextuals_svk, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_CRS
)

model2_svk = CNLS.CNLS(
    y = z_output_svk, 
    x = inputs_svk, 
    z = contextuals_svk, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_VRS
)

# define the CNLSDDF model with undesirable output
model3_svk = CNLSDDF.CNLSDDF(
    y = d_outputs_svk, 
    x = inputs_svk, 
    b = undesirables_svk, 
    gb = -1.0,
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)

# define the CNLSDDF model with undesirable output
model4_svk = CNLSDDF.CNLSDDF(
    y = d_outputs_svk, 
    x = inputs_svk, 
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)

In [5]:
model1_svk.optimize(OPT_LOCAL)
model2_svk.optimize(OPT_LOCAL)
model3_svk.optimize(OPT_LOCAL)
model4_svk.optimize(OPT_LOCAL)

Optimizing locally.
Estimating the additive model locally with mosek solver.
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : QO (quadratic optimization problem)
  Constraints            : 19881           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 989             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.02    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 141
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.

In [6]:
### inefficiency
ineff1_svk = StoNED.StoNED(model1_svk)
ineff2_svk = StoNED.StoNED(model2_svk)
ineff3_svk = StoNED.StoNED(model3_svk)
ineff4_svk = StoNED.StoNED(model4_svk)

In [7]:
res_svk = {
    "res1_svk" : {
        "alpha" : "None",
        "beta" : model1_svk.get_beta(),
        "delta" : "None",
        "gamma" : "None",
        "lamda" : model1_svk.get_lamda(),
        "residuals" : model1_svk.get_residual(),
        "frontier" : model1_svk.get_frontier(),
        "unconditional_expected_inefficiency" : ineff1_svk.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff1_svk.get_technical_inefficiency(method = RED_QLE)
    },


    "res2_svk" : {
        "alpha" : model2_svk.get_alpha(),
        "beta" : model2_svk.get_beta(),
        "delta" : "None",
        "gamma" : "None",
        "lamda" : model2_svk.get_lamda(),
        "residuals" : model2_svk.get_residual(),
        "frontier" : model2_svk.get_frontier(),
        "unconditional_expected_inefficiency" : ineff2_svk.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff2_svk.get_technical_inefficiency(method = RED_QLE)
    },


    "res3_svk" : {
        "alpha" : model3_svk.get_alpha(),
        "beta" : model3_svk.get_beta(),
        "delta" : model3_svk.get_delta(),
        "gamma" : model3_svk.get_gamma(),
        "lamda" : "None",
        "residuals" : model3_svk.get_residual(),
        "frontier" : "None",
        "unconditional_expected_inefficiency" : ineff3_svk.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff3_svk.get_technical_inefficiency(method = RED_QLE)
    },


    "res4_svk" : {
        "alpha" : "None",
        "beta" : model4_svk.get_beta(),
        "delta" : "None",
        "gamma" : model4_svk.get_gamma(),
        "lamda" : "None",
        "residuals" : model4_svk.get_residual(),
        "frontier" : "None",
        "unconditional_expected_inefficiency" : ineff4_svk.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff4_svk.get_technical_inefficiency(method = RED_QLE)
    }
}


In [8]:
import pickle
file = 'res_svk'
open_file = open(file, "wb")

pickle.dump(res_svk, open_file)
open_file.close()


# open_file = open(file, "rb")
# res_svk = pickle.load(open_file)
# open_file.close()

